In [ ]:
import sophus as sp
import polymetis
from torchcontrol.transform import Rotation as R

import fairotag as frt
from realsense_wrapper import RealsenseAPI

Instantiate camera modules

In [ ]:
rs = RealsenseAPI()

cameras = []
for intrinsics in rs.get_intrinsics():
    camera = frt.CameraModule()
    camera.set_intrinsics(intrinsics)
    cameras.append(camera)

Instantiate scene

In [ ]:
scene = frt.Scene()

scene.add_frame("ee")
for i, camera in enumerate(cameras):
    scene.add_camera(f"camera_{i + 1}")

Register markers

In [ ]:
# Involved markers
TABLE_MARKER = (0, 0.1)
EE_MARKER = (12, 0.04)

# Register markers in cameras
for camera in cameras:
    for marker in [TABLE_MARKER, EE_MARKER]:
        camera.register_marker_size(marker[0], marker[1])

# Register markers in scene
scene.add_marker(TABLE_MARKER[0], length=TABLE_MARKER[1])
scene.add_marker(EE_MARKER[0], length=EE_MARKER[1], frame="ee")

Collect calibration data

In [ ]:
NUM_SAMPLES = 20
POS_SAMPLE_RANGE = 0.3
ORI_SAMPLE_RANGE = 0.5

# Initialize robot control
robot = polymetis.RobotInterface(ip=) #TODO
robot.go_home()
robot.move_ee_xyz(torch.Tensor([0, 0, -0.2])) # move ee to center of workspace
ee_pos_home, ee_quat_home = robot.pose_ee()

# Collect samples
ee_poses = []
marker_observations = []
for i in range(NUM_SAMPLES):
    # Sample joint positions & move there 
    ee_pos_delta = torch.empty(3).uniform_(-POS_SAMPLE_RANGE, POS_SAMPLE_RANGE)
    ee_ori_delta = torch.empty(3).uniform_(-POS_SAMPLE_RANGE, POS_SAMPLE_RANGE)
    ee_pos_sampled = ee_pos_home + ee_pos_delta
    ee_quat_sampled = (R.from_quat(ee_quat_home) * R.from_rotvec(ee_ori_delta)).as_quat()

    robot.set_ee_pose(pos=ee_pos_sampled, quat=ee_quat_sampled)

    # Capture robot states
    ee_pos, ee_quat = robot.pose_ee()
    ee_pose = frt.utils.xyz_quat_to_se3(ee_pos, ee_quat)
    ee_poses.append(ee_pose)

    # Capture camera observations
    imgs = rs.get_images()
    marker_obs = [camera.detect_markers(img) for camera, img in zip(cameras, imgs)]
    marker_observations.append(marker_obs)

Optimize & visualize results

In [ ]:
scene.calibrate_extrinsics(
    detected_markers_ls=marker_observations,
    frame_transforms_ls=[("world", "ee", ee_pose) for ee_pose in ee_poses],
    verbosity=1,
)

scene.visualize()